#**1 Import**

In [1]:
#Import generali
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import os
import shutil
import cv2
import numpy as np
import random
from random import randint
from PIL import Image
import tensorflow as tf
import pickle
from tqdm import tqdm
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Import usati per la classificazione
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression    #L1 regression, L2 regression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [3]:
#Import usati per la classificazione con reti neurali
from tensorflow import keras
from keras import layers
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPool2D, Dropout, BatchNormalization,Activation,InputLayer,GlobalAveragePooling2D, ZeroPadding2D,Lambda
from keras import Model

In [4]:
#Monto il contenuto di google drive su colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opencv-python pytesseract numpy

In [ ]:
!pip install pytesseract

In [ ]:
!apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
!tesseract --version

tesseract 4.1.1
 leptonica-1.82.0
  libgif 5.1.9 : libjpeg 8d (libjpeg-turbo 2.1.1) : libpng 1.6.37 : libtiff 4.3.0 : zlib 1.2.11 : libwebp 1.2.2 : libopenjp2 2.4.0
 Found AVX512BW
 Found AVX512F
 Found AVX2
 Found AVX
 Found FMA
 Found SSE
 Found libarchive 3.6.0 zlib/1.2.11 liblzma/5.2.5 bz2lib/1.0.8 liblz4/1.9.3 libzstd/1.4.8


In [ ]:
!apt-get install tesseract-ocr-eng
!apt-get install tesseract-ocr-ita

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-eng set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-ita
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 1,067 kB of archives.
After this operation, 2,717 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-ita all 1:4.00~git30-7274cfa-1.1 [1,067 kB]
Fetched 1,067 kB in 2s (621 kB/s)
Selecting previously unselected package tesseract-ocr-ita.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-ita_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-oc

Per convertire le etichette stringa nella variabile target in valori numerici

In [5]:
!pip install scikit-learn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

#**2 Classificazione con modelli di ML (senza data augmentation)**

Adesso diamo in input le features estratte sia da ResNet50 che da VGG16 a diversi modelli di classificazione e vediamo con quali di questi otteniamo l'accuracy più alta.

In [ ]:
FeatureExtractionVGG16 = pd.read_csv('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/ProgettoDisgrafia/features_extraction/FeatureExtractionVGG16.csv')
FeatureExtractionResNet50 = pd.read_csv('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/ProgettoDisgrafia/features_extraction/FeatureExtractionResNet50.csv')

In [ ]:
#Assegniamo alla X il dataframe con le features estratte eliminando la colonna contenente le labels(diag)

#Senza l'argomento inplace = True dobbiamo assegnare il risultato ottenuto perchè la viariabile originale non viene modificata
XResNet50 = FeatureExtractionResNet50.drop(['diag'], axis=1)

XVGG16 = FeatureExtractionVGG16.drop(['diag'], axis=1)

#la Y è uguale per entrambi i dataframe e in questo caso la prendiamo da ResNet50
Y = FeatureExtractionResNet50['diag']

##2.1 Classificazione con features estratte da ResNet50

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
XResNet50.drop(["Unnamed: 0"], axis=1, inplace=True)

###2.1.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 100
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(XResNet50, Y, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]


   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)

     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac


     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Training accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.98
Test accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.84 

Training accuracy of logistic_regression with penalty = l2 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.83 

Training accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.84
Test accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.79 



In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

Training accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.89
Test accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.82 

Training accuracy of logistic_regression with penalty = l1 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l1 and with C=100.00: 0.80 

Training accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.53
Test accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.51 



###2.1.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).

In [ ]:
RS_NUM = 100
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(XResNet50, Y, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train)

   train_ac += decision_tree.score(X_train,y_train)

   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train))
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.72



###2.1.3 Classificazione con Random Forest

---



Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XResNet50, Y, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)  #modello addestrato
   rnd_forest.fit(X_train, y_train)                              #rappresentazione mediante features

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.82
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.82
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.81


###2.1.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XResNet50, Y, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']


  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)

   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac


   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.81
Training accuracy of SVM classifier with kernel = poly : 0.79
Test accuracy of SVM classifier with kernel = poly : 0.73
Training accuracy of SVM classifier with kernel = rbf : 0.77
Test accuracy of SVM classifier with kernel = rbf : 0.72
Training accuracy of SVM classifier with kernel = sigmoid : 0.56
Test accuracy of SVM classifier with kernel = sigmoid : 0.51


##2.2 Classificazione con features estratte da VGG16

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
XVGG16.drop(["Unnamed: 0"], axis=1, inplace=True)

###2.2.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 100
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(XVGG16, Y, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]


   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)

     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac


     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Training accuracy of logistic_regression with penalty = l2 and with C=1.00: 1.00
Test accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.79 

Training accuracy of logistic_regression with penalty = l2 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.79 

Training accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.86
Test accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.73 



In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

Training accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.95
Test accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.76 

Training accuracy of logistic_regression with penalty = l1 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l1 and with C=100.00: 0.76 

Training accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.53
Test accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.51 



###2.2.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).

In [ ]:
RS_NUM = 100
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(XVGG16, Y, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train)

   train_ac += decision_tree.score(X_train,y_train)

   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train))
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.68



###2.2.3 Classificazione con Random Forest

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XVGG16, Y, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.69
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.69
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.67


###2.2.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XVGG16, Y, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']


  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)

   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac


   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.79
Training accuracy of SVM classifier with kernel = poly : 0.82
Test accuracy of SVM classifier with kernel = poly : 0.72
Training accuracy of SVM classifier with kernel = rbf : 0.78
Test accuracy of SVM classifier with kernel = rbf : 0.66
Training accuracy of SVM classifier with kernel = sigmoid : 0.55
Test accuracy of SVM classifier with kernel = sigmoid : 0.49


#**3 Classificazione (senza data augmentation) utilizzando Random Forest**

Adesso diamo in input le features estratte da OpenCV, DenseNet121 e InceptionV3 al modello di classificazione Random Forest e vediamo l'accuracy più alta.

In [ ]:
FeatureExtractionOpenCV = pd.read_csv('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/ProgettoDisgrafia/features_extraction/FeatureExtractionOpenCV.csv')
FeatureExtractionDenseNet121 = pd.read_csv('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/ProgettoDisgrafia/features_extraction/FeatureExtractionDenseNet121.csv')
FeatureExtractionInceptionV3 = pd.read_csv('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/ProgettoDisgrafia/features_extraction/FeatureExtractionInceptionV3.csv')
FeatureExtractionAll = pd.read_csv('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/ProgettoDisgrafia/features_extraction/AllFeatureExtractionCombined2.csv')

In [ ]:
#Assegno alla X il dataframe con le features estratte eliminando la colonna contenente le labels(diag)

#Senza l'argomento inplace = True devo assegnare il risultato ottenuto perchè la viariabile originale non viene modificata
XOpenCV = FeatureExtractionOpenCV.drop(['diag'], axis=1)
XDenseNet121 = FeatureExtractionDenseNet121.drop(['diag'], axis=1)
XInceptionV3 = FeatureExtractionInceptionV3.drop(['diag'], axis=1)
XAll = FeatureExtractionAll.drop(['diag'], axis=1)

#la Y è uguale per i dataframe e in questo caso la prendiamo da OpenCV
Y1 = FeatureExtractionOpenCV['diag']
Y2 = FeatureExtractionDenseNet121['diag']
Y3 = FeatureExtractionInceptionV3['diag']

Y4 = FeatureExtractionAll['diag']



##3.1 Classificazione con features estratte da OpenCV



In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
XOpenCV.drop(["Unnamed: 0"], axis=1, inplace=True)

###3.1.1 Classificazione con Random Forest

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XOpenCV, Y1, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)  #modello addestrato
   rnd_forest.fit(X_train, y_train)                              #rappresentazione mediante features

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.69
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.69
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.68


##3.2 Classificazione con features estratte da DenseNet121

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
XDenseNet121.drop(["Unnamed: 0"], axis=1, inplace=True)

###3.2.1 Classificazione con Random Forest

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XDenseNet121, Y2, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)  #modello addestrato
   rnd_forest.fit(X_train, y_train)                              #rappresentazione mediante features

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.79
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.80
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.79


##3.3 Classificazione con features estratte da InceptionV3

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
XInceptionV3.drop(["Unnamed: 0"], axis=1, inplace=True)

###3.3.1 Classificazione con Random Forest

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XInceptionV3, Y3, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)  #modello addestrato
   rnd_forest.fit(X_train, y_train)                              #rappresentazione mediante features

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.78
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.78
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.79


##3.4 Classificazione con features estratte dalla combinazione totale

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
XAll.drop(["Unnamed: 0"], axis=1, inplace=True)

###3.4.1 Classificazione con Random Forest

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100

if 'DYSGR' in XAll.columns:
    XAll.drop('DYSGR', axis=1, inplace=True)

for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XAll, Y4, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)  #modello addestrato

   #Converti tutte le colonne in X_train in numeriche, sostituendo i valori non convertibili con NaN
   X_train = X_train.apply(pd.to_numeric, errors='coerce')

   #Elimino le righe con valori NaN in X_train e y_train
   X_train.dropna(inplace=True)
   indices = X_train.index
   y_train = y_train.loc[indices]  #mantieni solo i valori di y_train corrispondenti agli indici in X_train

   rnd_forest.fit(X_train, y_train)                              #rappresentazione mediante features

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac

   #Converti tutte le colonne in X_test in numeriche, sostituendo i valori non convertibili con NaN
   X_test = X_test.apply(pd.to_numeric, errors='coerce')

   #Elimino le righe con valori NaN in X_test e y_test
   X_test.dropna(inplace=True)
   indices = X_test.index
   y_test = y_test.loc[indices]

   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 1.00
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 1.00
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 1.00


##3.5 Addestramento e salvataggio :del modello RandomForest su tutti i dati disponibili


In [ ]:
# Addestro un modello finale su tutti i dati con n_estimators=100, da usare nell'interfaccia
import joblib

# Pulizia dei dati
XAll_clean = XAll.apply(pd.to_numeric, errors='coerce')
XAll_clean.dropna(inplace=True)
Y_clean = Y4.loc[XAll_clean.index]

# Addestramento del modello finale
final_model = RandomForestClassifier(n_estimators=100)
final_model.fit(XAll_clean, Y_clean)

# Salvataggio del modello in formato .pkl
joblib.dump(final_model, 'random_forest_model.pkl')
print(" Modello finale salvato come 'random_forest_model.pkl'")


 Modello finale salvato come 'random_forest_model.pkl'


Download in locale del modello RandomForest

In [ ]:
from google.colab import files
files.download('random_forest_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#**4 Classificazione con modelli di ML (con data augmentation)**

In [ ]:
FeatureExtractionDataAugmentationVGG16 = pd.read_csv('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/ProgettoDisgrafia/features_extraction/FeatureExtractionDataAugmentationVGG16.csv')
FeatureExtractionDataAugmentationResNet50 = pd.read_csv('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/ProgettoDisgrafia/features_extraction/FeatureExtractionDataAugmentationResNet50.csv')

In [ ]:
#Assegniamo alla X il dataframe con le features estratte eliminando la colonna contenente le labels(diag)

#Senza l'argomento inplace = True dobbiamo assegnare il risultato ottenuto perchè la viariabile originale non viene modificata
X_ResNet50 = FeatureExtractionDataAugmentationResNet50.drop(['diag'], axis=1)

X_VGG16 = FeatureExtractionDataAugmentationVGG16.drop(['diag'], axis=1)

#la Y è uguale per entrambi i dataframe e in questo caso la prendiamo da ResNet50
Y_da = FeatureExtractionDataAugmentationResNet50['diag']

##4.1 Classificazione con features estratte da ResNet50

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
X_ResNet50.drop(["Unnamed: 0"], axis=1, inplace=True)

###4.1.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 100
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X_ResNet50, Y_da, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]


   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)

     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac


     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Training accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.97
Test accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.80 

Training accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.98
Test accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.78 

Training accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.83
Test accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.79 



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

Training accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.93
Test accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.79 

Training accuracy of logistic_regression with penalty = l1 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l1 and with C=100.00: 0.76 

Training accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.66
Test accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.62 



###4.1.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).

In [ ]:
RS_NUM = 100
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X_ResNet50, Y_da, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train)

   train_ac += decision_tree.score(X_train,y_train)

   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train))
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.65



###4.1.3 Classificazione con Random Forest

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_ResNet50, Y_da, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.78
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.77
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.77


###4.1.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_ResNet50, Y_da, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']


  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)

   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac


   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.74
Training accuracy of SVM classifier with kernel = poly : 0.80
Test accuracy of SVM classifier with kernel = poly : 0.75
Training accuracy of SVM classifier with kernel = rbf : 0.79
Test accuracy of SVM classifier with kernel = rbf : 0.74
Training accuracy of SVM classifier with kernel = sigmoid : 0.74
Test accuracy of SVM classifier with kernel = sigmoid : 0.71


##4.2 Classificazione con features estratte da VGG16

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
X_VGG16.drop(["Unnamed: 0"], axis=1, inplace=True)

###4.2.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 100
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X_VGG16, Y_da, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]


   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)

     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac


     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Training accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.99
Test accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.80 

Training accuracy of logistic_regression with penalty = l2 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.78 

Training accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.86
Test accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.79 



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

Training accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.98
Test accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.77 

Training accuracy of logistic_regression with penalty = l1 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l1 and with C=100.00: 0.78 

Training accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.53
Test accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.49 



###4.2.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).

In [ ]:
RS_NUM = 100
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X_VGG16, Y_da, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train)

   train_ac += decision_tree.score(X_train,y_train)

   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train))
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.63



###4.2.3 Classificazione con Random Forest

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_VGG16, Y_da, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.74
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.74
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.72


###4.2.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_VGG16, Y_da, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']


  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)

   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac


   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.77
Training accuracy of SVM classifier with kernel = poly : 0.83
Test accuracy of SVM classifier with kernel = poly : 0.76
Training accuracy of SVM classifier with kernel = rbf : 0.79
Test accuracy of SVM classifier with kernel = rbf : 0.73
Training accuracy of SVM classifier with kernel = sigmoid : 0.64
Test accuracy of SVM classifier with kernel = sigmoid : 0.63


#**5 Classificazione (con data augmentation) utilizzando Random Forest**

In [ ]:
FeatureExtractionDataAugmentationOpenCV = pd.read_csv('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/ProgettoDisgrafia/features_extraction/FeatureExtractionDataAugmentationOpenCV.csv')
FeatureExtractionDataAugmentationDenseNet121 = pd.read_csv('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/ProgettoDisgrafia/features_extraction/FeatureExtractionDataAugmentationDenseNet121.csv')
FeatureExtractionDataAugmentationInceptionV3 = pd.read_csv('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/ProgettoDisgrafia/features_extraction/FeatureExtractionDataAugmentationInceptionV3.csv')

In [ ]:
#Assegniamo alla X il dataframe con le features estratte eliminando la colonna contenente le labels(diag)

#Senza l'argomento inplace = True dobbiamo assegnare il risultato ottenuto perchè la viariabile originale non viene modificata
X_OpenCV = FeatureExtractionDataAugmentationOpenCV.drop(['diag'], axis=1)
X_DenseNet121 = FeatureExtractionDataAugmentationDenseNet121.drop(['diag'], axis=1)
X_InceptionV3 = FeatureExtractionDataAugmentationInceptionV3.drop(['diag'], axis=1)

Y_da1 = FeatureExtractionDataAugmentationOpenCV['diag']
Y_da2 = FeatureExtractionDataAugmentationDenseNet121['diag']
Y_da3 = FeatureExtractionDataAugmentationInceptionV3['diag']

##5.1 Classificazione con features estratte da OpenCV

In [ ]:
#Rimuovo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
X_OpenCV.drop(["Unnamed: 0"], axis=1, inplace=True)

###5.1.1 Classificazione con Random Forest

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_OpenCV, Y_da1, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.68
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.66
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.67


##5.2 Classificazione con features estratte da DenseNet121

In [ ]:
#Rimuovo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
X_DenseNet121.drop(["Unnamed: 0"], axis=1, inplace=True)

###5.2.1 Classificazione con Random Forest

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_DenseNet121, Y_da2, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.74
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.74
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.74


##5.3 Classificazione con features estratte da InceptionV3

In [ ]:
#Rimuovo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
X_InceptionV3.drop(["Unnamed: 0"], axis=1, inplace=True)

###5.3.1 Classificazione con Random Forest

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_InceptionV3, Y_da3, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.77
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.77
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.75


#**6 Concatenazione delle features ottenute senza data augmentation e con data augmentation**

Applichiamo questo procedimento solo a ResNet50

In [ ]:
#Otteniamo le features estratte da ResNet50 con data augmentation e senza data augmentation
FeatureExtractionResNet50 = pd.read_csv('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/ProgettoDisgrafia/features_extraction/FeatureExtractionResNet50.csv')
FeatureExtractionDataAugmentationResNet50 = pd.read_csv('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/ProgettoDisgrafia/features_extraction/FeatureExtractionDataAugmentationResNet50.csv')


In [ ]:
#Funzione `concat` per concatenare i DataFrame
concat_dataframe = pd.concat([FeatureExtractionResNet50, FeatureExtractionDataAugmentationResNet50])
concat_dataframe.drop(["Unnamed: 0"], axis=1, inplace=True)

In [ ]:
concat_dataframe.to_csv('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/ProgettoDisgrafia/features_extraction/ConcatFeatureResNet50.csv')

In [ ]:
X = concat_dataframe.drop(['diag'], axis=1)

Y = concat_dataframe['diag']

###6.1.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 10
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]


   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)

     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac


     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Training accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.97
Test accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.91 

Training accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.98
Test accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.88 

Training accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.84
Test accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.84 



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

Training accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.94
Test accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.80 

Training accuracy of logistic_regression with penalty = l1 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l1 and with C=100.00: 0.88 

Training accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.53
Test accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.52 



###6.1.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).

In [ ]:
RS_NUM = 10
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train)

   train_ac += decision_tree.score(X_train,y_train)

   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train))
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.77



###6.1.3 Classificazione con Random Forest

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 10
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]


  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)

   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac


   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.84
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.85
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.83


###6.1.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica).
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 10
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']


  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)

   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac


   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.80
Training accuracy of SVM classifier with kernel = poly : 0.81
Test accuracy of SVM classifier with kernel = poly : 0.76
Training accuracy of SVM classifier with kernel = rbf : 0.80
Test accuracy of SVM classifier with kernel = rbf : 0.74
Training accuracy of SVM classifier with kernel = sigmoid : 0.75
Test accuracy of SVM classifier with kernel = sigmoid : 0.70


#**7 Classificazione con un modello preaddestrato di InceptionV3**

In [ ]:
directory = '/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/file/DatasetLabel1'
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    image_size=(432, 288),
    seed=123,
    validation_split= 0.2,
    subset='training',
)

Found 360 files belonging to 2 classes.
Using 288 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    image_size=(432, 288),
    seed=123,
    validation_split= 0.2,
    subset='validation',
)

Found 360 files belonging to 2 classes.
Using 72 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['Dysgraphic', 'No_Dysgraphic']


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout #Importa Dropout

InceptionV3 = InceptionV3(include_top=False, input_shape=(432, 288, 3), weights="imagenet")

# Non è necessario congelare i livelli quando si utilizza il modello solo per l'estrazione di feature
# for layer in InceptionV3.layers:
#     layer.trainable = False

model = tf.keras.Sequential()

model.add(InceptionV3)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu')) # livello Dense prima di Dropout
model.add(Dropout(0.5)) # Ora Dropout conosce la forma di input
model.add(Dense(1, activation='sigmoid'))
# Viene rimosso il secondo livello Dropout dopo il livello Dense finale

#  Viene costruito il modello con un input di esempio per definire le forme
model.build(input_shape=(None, 432, 288, 3))


In [ ]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)            │ (None, 12, 7, 2048)         │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_4           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 256)                 │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,327,585 (85.17 MB)

 Trainable params: 22,293,153 (85.04 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=200
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 190s 9s/step - accuracy: 0.5263 - loss: 0.9609 - val_accuracy: 0.4583 - val_loss: 483.9626
Epoch 2/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 53s 613ms/step - accuracy: 0.7404 - loss: 0.5606 - val_accuracy: 0.5417 - val_loss: 701.6800
Epoch 3/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 636ms/step - accuracy: 0.8019 - loss: 0.4419 - val_accuracy: 0.5417 - val_loss: 909.6672
Epoch 4/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 619ms/step - accuracy: 0.7717 - loss: 0.5557 - val_accuracy: 0.5417 - val_loss: 27.5002
Epoch 5/200
1/9 ━━━━━━━━━━━━━━━━━━━━ 42s 5s/step - accuracy: 0.7812 - loss: 0.5284

KeyboardInterrupt: 

In [ ]:
# scongeliamo i pesi dei layer del modello
InceptionV3.trainable = True


model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=[keras.metrics.BinaryAccuracy()])


history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=200
)

Epoch 1/200


KeyboardInterrupt: 

#**8 Classificazione con rete neurale pre-addestra sul Dataset IAM**

Il dataset IAM contiene forme di testo scritte a mano in inglese.

Il modello e i relativi pesi pre-addestrati sono stati presi da qui:
https://github.com/diegocasmo/iam_writer_recognition


In [ ]:

model = Sequential()

# Define network input shape
model.add(ZeroPadding2D((1, 1), input_shape=(432, 288, 1)))

# CNN model - Building the model suggested in paper
model.add(Conv2D(filters= 32, kernel_size =(5,5), strides= (2, 2), padding='same', name='conv1'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool1'))

model.add(Conv2D(filters= 64, kernel_size =(3, 3), strides= (1, 1), padding='same', name='conv2'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool2'))

model.add(Conv2D(filters= 128, kernel_size =(3, 3), strides= (1, 1), padding='same', name='conv3'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool3'))


model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(512, name='dense1'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256, name='dense2'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/zero_padding2d.py:72: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.load_weights('/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/Progetto-ML-DL-main/model_weights.hdf5',by_name=True,skip_mismatch=True)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_api.py:262: UserWarning: Skipping loading weights for layer #13 (named dense1) due to mismatch in shape for weight sequential_5/dense1/kernel. Weight expects shape (62208, 512). Received saved weight with shape (1152, 512)
  legacy_h5_format.load_weights_from_hdf5_group_by_name(


In [ ]:
model.trainable=False

In [ ]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ zero_padding2d (ZeroPadding2D)       │ (None, 434, 290, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1 (Conv2D)                       │ (None, 217, 145, 32)        │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_470 (Activation)          │ (None, 217, 145, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool1 (MaxPooling2D)                 │ (None, 108, 72, 32)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2 (Conv2D)                       │ (None, 108, 72, 64)         │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_471 (Activation)          │ (None, 108, 72, 64)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool2 (MaxPooling2D)                 │ (None, 54, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3 (Conv2D)                       │ (None, 54, 36, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_472 (Activation)          │ (None, 54, 36, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pool3 (MaxPooling2D)                 │ (None, 27, 18, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 62208)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 62208)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense1 (Dense)                       │ (None, 512)                 │      31,851,008 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_473 (Activation)          │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense2 (Dense)                       │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_474 (Activation)          │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 256)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 32,075,520 (122.36 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 32,075,520 (122.36 MB)

In [ ]:
new_model = Sequential()

new_model.add(model)


new_model.add(Dense(1, name='output'))
new_model.add(Activation('sigmoid'))

In [ ]:
new_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_5 (Sequential)            │ (None, 256)                 │      32,075,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_475 (Activation)          │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 32,075,520 (122.36 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 32,075,520 (122.36 MB)

In [ ]:
directory = '/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/file/DatasetLabel1'
train = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(432, 288),
    seed=123,
    validation_split= 0.2,
    subset='training',
)

Found 360 files belonging to 2 classes.
Using 288 files for training.


In [ ]:
valid = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(432, 288),
    seed=123,
    validation_split= 0.2,
    subset='validation',
)

Found 360 files belonging to 2 classes.
Using 72 files for validation.


In [ ]:
new_model.compile(optimizer=Adam(learning_rate = 0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=200
history = new_model.fit(
  train,
  validation_data=valid,
  epochs=epochs
)

Epoch 1/200
5/9 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5089 - loss: 23.1425

KeyboardInterrupt: 

In [ ]:
# scongeliamo i pesi dei layer del modello
model.trainable = True


new_model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])


history = new_model.fit(
  train,
  validation_data=valid,
  epochs=200
)

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


9/9 [==============================] - 34s 3s/step - loss: 0.8873 - binary_accuracy: 0.5486 - val_loss: 0.6807 - val_binary_accuracy: 0.5139
Epoch 2/200
9/9 [==============================] - 30s 3s/step - loss: 0.7472 - binary_accuracy: 0.5590 - val_loss: 0.6813 - val_binary_accuracy: 0.5278
Epoch 3/200
9/9 [==============================] - 31s 3s/step - loss: 0.8042 - binary_accuracy: 0.5139 - val_loss: 0.6849 - val_binary_accuracy: 0.5139
Epoch 4/200
9/9 [==============================] - 30s 3s/step - loss: 0.7971 - binary_accuracy: 0.5521 - val_loss: 0.6929 - val_binary_accuracy: 0.5000
Epoch 5/200
9/9 [==============================] - 33s 4s/step - loss: 0.6940 - binary_accuracy: 0.5938 - val_loss: 0.6912 - val_binary_accuracy: 0.5000
Epoch 6/200
9/9 [==============================] - 29s 3s/step - loss: 0.6990 - binary_accuracy: 0.6111 - val_loss: 0.6829 - val_binary_accuracy: 0.5139
Epoch 7/200
9/9 [==============================] - 31s 3s/step - loss: 0.7354 - binary_accurac

#**9 Text Detection**

In [ ]:
import cv2
import pytesseract
import numpy as np

# 1. Rilevamento delle frasi usando Tesseract OCR
def detect_text_sentences(image_path):
    # Carica l'immagine
    image = cv2.imread(image_path)
    if image is None:
        return "L'immagine non è stata caricata correttamente"

    # Esegui il rilevamento delle frasi
    d = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)

    return d

# 2. Estrazione e salvataggio delle frasi come immagini separate
def save_text_sentences_as_images(image, data, output_directory,base_filename):
    num_boxes = len(data['text'])

    for i in range(num_boxes):
        if int(data['conf'][i]) > 40 and data['text'][i].strip():  # Ignora i box con bassa fiducia
            x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]

            # Estrai la frase dall'immagine
            sentence_region = image[y:y+h, x:x+w]
            # Verifica se l'immagine estratta è valida (non vuota)
            if not sentence_region.size:
                continue
            # Salva la frase come immagine
            output_path = f"{output_directory}/{base_filename}_sentence_{i}.png"
            cv2.imwrite(output_path, sentence_region)
            print(f"Frase {i} salvata come immagine: {output_path}")

In [ ]:
import os

def save_image(dataset_folder,output_directory):
  # Elabora tutte le immagini nel dataset
  for filename in os.listdir(dataset_folder):
          image_path = os.path.join(dataset_folder, filename)

          # 1. Rilevamento delle frasi
          text_data = detect_text_sentences(image_path)

          # 2. Estrazione e salvataggio delle frasi come immagini separate
          image = cv2.imread(image_path)
          base_filename, _ = os.path.splitext(filename)
          save_text_sentences_as_images(image, text_data, output_directory,base_filename)


In [ ]:
# Esempio di utilizzo della funzione di text detection e salvataggio delle nuove immagini
dataset_DISG = '/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/file/datasetLabel/Dysgraphic/'
dataset_NO_DISG = '/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/file/datasetLabel/No_Dysgraphic/'

save_image(dataset_DISG,'/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/file/textDetection/Dysgraphic')
save_image(dataset_NO_DISG,'/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/file/textDetection/No_Dysgraphic')

Frase 10 salvata come immagine: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic/user00128_sentence_10.png
Frase 16 salvata come immagine: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic/user00128_sentence_16.png
Frase 18 salvata come immagine: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic/user00128_sentence_18.png
Frase 33 salvata come immagine: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic/user00128_sentence_33.png
Frase 9 salvata come immagine: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic/user00141_sentence_9.png
Frase 14 salvata come immagine: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic/user00141_sentence_14.png
Frase 25 salvata come immagine: /content/drive/MyDrive/ProgettoML/All datasets/file/textDetection/Dysgraphic/user00141_sentence_25.png
Frase 26 salvata come immagine: /content/drive/MyDrive/Pr

In [ ]:
import os
from PIL import Image

def resize_image(image, target_height):
    aspect_ratio = target_height / float(image.height)
    target_width = int(image.width * aspect_ratio)
    resized_image = image.resize((target_width, target_height))
    return resized_image

def pad_image(image, target_width, target_height):
    pad_width = max(0, target_width - image.width)
    pad_height = max(0, target_height - image.height)
    padded_image = Image.new(image.mode, (target_width, target_height), (255, 255, 255))
    padded_image.paste(image, ((pad_width // 2), (pad_height // 2)))
    return padded_image

def process_images(input_folder, output_folder, target_height, target_width):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            original_image = Image.open(input_path)
            resized_image = resize_image(original_image, target_height)
            padded_image = pad_image(resized_image, target_width, target_height)
            padded_image.save(output_path)

    print("Processo completato. Immagini modificate salvate in:", output_folder)

# Percorsi delle cartelle di input e output
input_folder = '/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/file/textDetection/Dysgraphic'
output_folder = '/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/file/textDetection/Dysgraphic'

input_folder1 = '/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/file/textDetection/No_Dysgraphic'
output_folder1 = '/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/file/textDetection/No_Dysgraphic'

target_height = 75  # Sostituisci con l'altezza desiderata
target_width = 299  # Larghezza richiesta per InceptionV3

# Processo le immagini
process_images(input_folder, output_folder, target_height, target_width)
process_images(input_folder1, output_folder1, target_height, target_width)


Processo completato. Immagini modificate salvate in: /content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta/file/textDetection/Dysgraphic
Processo completato. Immagini modificate salvate in: /content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta/file/textDetection/No_Dysgraphic


In [ ]:
directory = '/content/drive/MyDrive/2023-DL-progetto-Marchi-Pallotta-Fadale/file/textDetection'
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    image_size=(75, 299),
    seed=123,
    validation_split= 0.2,
    subset='training',
)

Found 868 files belonging to 2 classes.
Using 695 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    image_size=(75, 299),
    seed=123,
    validation_split= 0.2,
    subset='validation',
)

Found 868 files belonging to 2 classes.
Using 173 files for validation.


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout, Input

# Definisci una forma di input
input_shape = (150, 299, 3)  # O (299, 299, 3)

# Crea il modello InceptionV3 con la forma di input specificata
input_tensor = Input(shape=input_shape)  # Aggiungi un livello di input
base_model = InceptionV3(include_top=False, input_tensor=input_tensor, weights="imagenet")

# Blocca i layer del modello base
for layer in base_model.layers:
    layer.trainable = False

# Crea il tuo modello
model = tf.keras.Sequential(layers=[
    Input(shape=input_shape), # Aggiungi un livello di input con la forma specificata
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [ ]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)            │ (None, 3, 8, 2048)          │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_8           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 256)                 │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,327,585 (85.17 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=200
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/200
22/22 [==============================] - 127s 3s/step - loss: 62.5677 - accuracy: 0.5079 - val_loss: 22.6464 - val_accuracy: 0.5145
Epoch 2/200
22/22 [==============================] - 2s 68ms/step - loss: 32.8652 - accuracy: 0.5094 - val_loss: 3.4884 - val_accuracy: 0.5202
Epoch 3/200
22/22 [==============================] - 2s 61ms/step - loss: 10.5255 - accuracy: 0.5223 - val_loss: 1.0530 - val_accuracy: 0.5087
Epoch 4/200
22/22 [==============================] - 2s 91ms/step - loss: 2.7768 - accuracy: 0.5165 - val_loss: 0.7147 - val_accuracy: 0.4971
Epoch 5/200
22/22 [==============================] - 2s 74ms/step - loss: 1.1360 - accuracy: 0.5180 - val_loss: 0.7080 - val_accuracy: 0.5087
Epoch 6/200
22/22 [==============================] - 2s 61ms/step - loss: 0.8578 - accuracy: 0.5468 - val_loss: 0.7057 - val_accuracy: 0.5145
Epoch 7/200
22/22 [==============================] - 2s 65ms/step - loss: 0.9233 - accuracy: 0.5122 - val_loss: 0.7061 - val_accuracy: 0.5087
Ep

In [ ]:
# scongeliamo i pesi dei layer del modello
InceptionV3.trainable = True


model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=[keras.metrics.BinaryAccuracy()])


history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=200
)

Epoch 1/200
22/22 [==============================] - 39s 188ms/step - loss: 0.0102 - binary_accuracy: 1.0000 - val_loss: 1.3102 - val_binary_accuracy: 0.6763
Epoch 2/200
22/22 [==============================] - 3s 128ms/step - loss: 0.0114 - binary_accuracy: 0.9986 - val_loss: 1.3465 - val_binary_accuracy: 0.6705
Epoch 3/200
22/22 [==============================] - 4s 148ms/step - loss: 0.0072 - binary_accuracy: 1.0000 - val_loss: 1.3287 - val_binary_accuracy: 0.6705
Epoch 4/200
22/22 [==============================] - 3s 136ms/step - loss: 0.0066 - binary_accuracy: 1.0000 - val_loss: 1.3822 - val_binary_accuracy: 0.6763
Epoch 5/200
22/22 [==============================] - 3s 130ms/step - loss: 0.0072 - binary_accuracy: 1.0000 - val_loss: 1.4459 - val_binary_accuracy: 0.6705
Epoch 6/200
22/22 [==============================] - 3s 130ms/step - loss: 0.0073 - binary_accuracy: 1.0000 - val_loss: 1.4394 - val_binary_accuracy: 0.6879
Epoch 7/200
22/22 [==============================] - 4s 1